## **Data Loading and Preprocessing**

In [1]:
# Importing Libraries
import pandas as pd

# Load the MovieLens 1M Dataset
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip

ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', engine='python',
                      names=['UserID', 'MovieID', 'Rating', 'Timestamp'], encoding='latin1')
movies = pd.read_csv('ml-1m/movies.dat', sep='::', engine='python',
                     names=['MovieID', 'Title', 'Genres'], encoding='latin1')
users = pd.read_csv('ml-1m/users.dat', sep='::', engine='python',
                    names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], encoding='latin1')

# Merge the tables
merged_data = pd.merge(pd.merge(ratings, movies, on='MovieID'), users, on='UserID')

merged_data.head()



--2024-04-08 14:46:39--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  18.9MB/s    in 0.3s    

2024-04-08 14:46:40 (18.9 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


## **Preparing Data for Surprise**

In [2]:
!pip install surprise
# Importing the Dataset and Reader classes from the Surprise library. The Dataset class is used to load datasets, and the Reader class is used to parse rating data.
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

# Preparing Data for Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_data[['UserID', 'MovieID', 'Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162993 sha256=cc4b8c2c74a1c7fb337ef02f694b511b73f0c7e6b6f22f210d9ae7015dfd6074
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


## **Training Collaborative Filtering Algorithm**

In [3]:
!pip install scikit-surprise
from surprise import SVD

# This class is used to load datasets into a format that can be used by the scikit-surprise library
from surprise import Dataset

trainset = data.build_full_trainset()

# Training Collaborative Filtering Algorithm.
algo = SVD()
algo.fit(trainset)


## **Making Predictions and Calculating RMSE**

In [4]:
from sklearn.metrics import mean_squared_error
from math import sqrt

# Making Predictions for the test set using the trained collaborative filtering algorithm (algo). The testset contains user-item pairs for which we want to predict ratings.
predictions = algo.test(testset)

# Calculating RMSE using the predictions.
# First, creating two lists: one containing the actual ratings (r_ui) and the other containing the estimated ratings (est) for each prediction. Then, calculating the mean squared error between these two lists and takes the square root to obtain the RMSE
rmse = sqrt(mean_squared_error([pred.r_ui for pred in predictions], [pred.est for pred in predictions]))
print("RMSE:", rmse)


RMSE: 0.6721692345917637
